# Decision Trees

## Import

In [1]:
import graphlab
graphlab.canvas.set_target('ipynb')

In [ ]:
loans = graphlab.SFrame('lending-club-data.gl/')

In [10]:
loans[0: 1]

Columns:
	id	int
	member_id	int
	loan_amnt	int
	funded_amnt	int
	funded_amnt_inv	int
	term	str
	int_rate	float
	installment	float
	grade	str
	sub_grade	str
	emp_title	str
	emp_length	str
	home_ownership	str
	annual_inc	int
	is_inc_v	str
	issue_d	str
	loan_status	str
	pymnt_plan	str
	url	str
	desc	str
	purpose	str
	title	str
	zip_code	str
	addr_state	str
	dti	float
	delinq_2yrs	int
	earliest_cr_line	str
	inq_last_6mths	int
	mths_since_last_delinq	int
	mths_since_last_record	int
	open_acc	int
	pub_rec	int
	revol_bal	int
	revol_util	float
	total_acc	int
	initial_list_status	str
	out_prncp	float
	out_prncp_inv	float
	total_pymnt	float
	total_pymnt_inv	float
	total_rec_prncp	float
	total_rec_int	float
	total_rec_late_fee	float
	recoveries	float
	collection_recovery_fee	float
	last_pymnt_d	str
	last_pymnt_amnt	float
	next_pymnt_d	str
	last_credit_pull_d	str
	collections_12_mths_ex_med	int
	mths_since_last_major_derog	str
	policy_code	int
	not_compliant	int
	status	str
	inactive_loans	int
	bad_loans	int
	emp_length_num	int
	grade_num	int
	sub_grade_num	float
	delinq_2yrs_zero	int
	pub_rec_zero	int
	collections_12_mths_zero	int
	short_emp	int
	payment_inc_ratio	float
	final_d	str
	last_delinq_none	int
	last_record_none	int
	last_major_derog_none	int

Rows: 1

Data:
+---------+-----------+-----------+-------------+-----------------+------------+
|    id   | member_id | loan_amnt | funded_amnt | funded_amnt_inv |    term    |
+---------+-----------+-----------+-------------+-----------------+------------+
| 1077501 |  1296599  |    5000   |     5000    |       4975      |  36 months |
+---------+-----------+-----------+-------------+-----------------+------------+
+----------+-------------+-------+-----------+-----------+------------+----------------+
| int_rate | installment | grade | sub_grade | emp_title | emp_length | home_ownership |
+----------+-------------+-------+-----------+-----------+------------+----------------+
|  10.65   |    162.87   |   B   |     B2    |           | 10+ years  |      RENT      |
+----------+-------------+-------+-----------+-----------+------------+----------------+
+------------+----------+-----------------+-------------+------------+
| annual_inc | is_inc_v |     issue_d     | loan_status | pymnt_plan |
+------------+----------+-----------------+-------------+------------+
|   24000    | Verified | 20111201T000000 |  Fully Paid |     n      |
+------------+----------+-----------------+-------------+------------+
+-------------------------------+-------------------------------+-----+
|              url              |              desc             | ... |
+-------------------------------+-------------------------------+-----+
| https://www.lendingclub.co... |   Borrower added on 12/22/... | ... |
+-------------------------------+-------------------------------+-----+
[1 rows x 68 columns]

## Data exploration

In [3]:
loans.column_names()

['id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'is_inc_v',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'not_compliant',
 'status',
 'inactive_loans',
 'bad_loans',
 'emp_length_num',
 'grade_num',
 'sub_grade_num',
 'delinq_2yrs_zero',
 'pub_rec

In [4]:
loans['grade'].show()

In [5]:
loans['home_ownership'].show()

### Transform target

In [11]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x == 0 else -1)
loans = loans.remove_column('bad_loans')

In [12]:
loans['safe_loans'].show(view = 'Categorical')

### Features for the classification

In [13]:
features = ['grade',
            'sub_grade',
            'short_emp',
            'emp_length_num',
            'home_ownership',
            'dti',
            'purpose',
            'term',
            'last_delinq_none',
            'last_major_derog_none',
            'revol_util',
            'total_rec_late_fee',
           ]
target = 'safe_loans'
loans = loans[features + [target]]

In [15]:
loans[0:1]

Columns:
	grade	str
	sub_grade	str
	short_emp	int
	emp_length_num	int
	home_ownership	str
	dti	float
	purpose	str
	term	str
	last_delinq_none	int
	last_major_derog_none	int
	revol_util	float
	total_rec_late_fee	float
	safe_loans	int

Rows: 1

Data:
+-------+-----------+-----------+----------------+----------------+-------+
| grade | sub_grade | short_emp | emp_length_num | home_ownership |  dti  |
+-------+-----------+-----------+----------------+----------------+-------+
|   B   |     B2    |     0     |       11       |      RENT      | 27.65 |
+-------+-----------+-----------+----------------+----------------+-------+
+-------------+------------+------------------+-----------------------+------------+
|   purpose   |    term    | last_delinq_none | last_major_derog_none | revol_util |
+-------------+------------+------------------+-----------------------+------------+
| credit_card |  36 months |        1         |           1           |    83.7    |
+-------------+------------+------------------+-----------------------+------------+
+--------------------+------------+
| total_rec_late_fee | safe_loans |
+--------------------+------------+
|        0.0         |     1      |
+--------------------+------------+
[1 rows x 13 columns]

## Balance classes

In [16]:
safe_loans_raw = loans[loans[target] == +1]
risky_loans_raw = loans[loans[target] == -1]
print "Number of safe loans  : %s" % len(safe_loans_raw)
print "Number of risky loans : %s" % len(risky_loans_raw)

Number of safe loans  : 99457
Number of risky loans : 23150


In [21]:
print "Percentage of safe loans  :", (len(safe_loans_raw) * 100) / (len(safe_loans_raw) + len(risky_loans_raw))
print "Percentage of risky loans :", (len(risky_loans_raw) * 100) / (len(safe_loans_raw) + len(risky_loans_raw))

Percentage of safe loans  : 81
Percentage of risky loans : 18


In [22]:
percentage = len(risky_loans_raw)/float(len(safe_loans_raw))
risky_loans = risky_loans_raw
safe_loans = safe_loans_raw.sample(percentage, seed=1)
loans_data = risky_loans.append(safe_loans)

In [24]:
print "Percentage of safe loans                 :", len(safe_loans) / float(len(loans_data))
print "Percentage of risky loans                :", len(risky_loans) / float(len(loans_data))
print "Total number of loans in our new dataset :", len(loans_data)

Percentage of safe loans                 : 0.502236174422
Percentage of risky loans                : 0.497763825578
Total number of loans in our new dataset : 46508


## Split data

In [25]:
train_data, validation_data = loans_data.random_split(.8, seed = 1)

## Build model DTC

In [26]:
decision_tree_model = graphlab.decision_tree_classifier.create(train_data, validation_set = None, target = target, features = features)

Decision tree classifier:

--------------------------------------------------------

Number of examples          : 37224

Number of classes           : 2

Number of feature columns   : 12

...

+-----------+--------------+-------------------+-------------------+

| Iteration | Elapsed Time | Training-accuracy | Training-log_loss |

+-----------+--------------+-------------------+-------------------+

| 1         | 0.142274     | 0.640581          | 0.663259          |

+-----------+--------------+-------------------+-------------------+

## Visualize model

In [27]:
small_model = graphlab.decision_tree_classifier.create(train_data, validation_set = None, target = target, features = features, max_depth = 2)

Decision tree classifier:

--------------------------------------------------------

Number of examples          : 37224

Number of classes           : 2

Number of feature columns   : 12

...

+-----------+--------------+-------------------+-------------------+

| Iteration | Elapsed Time | Training-accuracy | Training-log_loss |

+-----------+--------------+-------------------+-------------------+

| 1         | 0.023065     | 0.613502          | 0.676098          |

+-----------+--------------+-------------------+-------------------+

In [28]:
small_model.show(view = "Tree")

## Make predictions

In [29]:
validation_safe_loans = validation_data[validation_data[target] == 1]
validation_risky_loans = validation_data[validation_data[target] == -1]

sample_validation_data_risky = validation_risky_loans[0:2]
sample_validation_data_safe = validation_safe_loans[0:2]

sample_validation_data = sample_validation_data_safe.append(sample_validation_data_risky)
sample_validation_data

Columns:
	grade	str
	sub_grade	str
	short_emp	int
	emp_length_num	int
	home_ownership	str
	dti	float
	purpose	str
	term	str
	last_delinq_none	int
	last_major_derog_none	int
	revol_util	float
	total_rec_late_fee	float
	safe_loans	int

Rows: 4

Data:
+-------+-----------+-----------+----------------+----------------+-------+
| grade | sub_grade | short_emp | emp_length_num | home_ownership |  dti  |
+-------+-----------+-----------+----------------+----------------+-------+
|   B   |     B3    |     0     |       11       |      OWN       | 11.18 |
|   D   |     D1    |     0     |       10       |      RENT      | 16.85 |
|   D   |     D2    |     0     |       3        |      RENT      | 13.97 |
|   A   |     A5    |     0     |       11       |    MORTGAGE    | 16.33 |
+-------+-----------+-----------+----------------+----------------+-------+
+--------------------+------------+------------------+-----------------------+
|      purpose       |    term    | last_delinq_none | last_major_derog_none |
+--------------------+------------+------------------+-----------------------+
|    credit_card     |  36 months |        1         |           1           |
| debt_consolidation |  36 months |        1         |           1           |
|       other        |  60 months |        0         |           1           |
| debt_consolidation |  36 months |        1         |           1           |
+--------------------+------------+------------------+-----------------------+
+------------+--------------------+------------+
| revol_util | total_rec_late_fee | safe_loans |
+------------+--------------------+------------+
|    82.4    |        0.0         |     1      |
|    96.4    |        0.0         |     1      |
|    59.5    |        0.0         |     -1     |
|    62.1    |        0.0         |     -1     |
+------------+--------------------+------------+
[4 rows x 13 columns]

### Explore predictions

In [46]:
predictions = decision_tree_model.predict(sample_validation_data)
predictions

dtype: int
Rows: 4
[1L, -1L, -1L, 1L]

In [51]:
print sum(sample_validation_data['safe_loans'] == predictions) * 100 / len(sample_validation_data)

50


## Explore probability predictions

In [53]:
decision_tree_model.predict(sample_validation_data, output_type = 'probability')

dtype: float
Rows: 4
[0.5473502278327942, 0.4891221821308136, 0.4559234082698822, 0.5864479541778564]

In [54]:
small_model.predict(sample_validation_data, output_type = 'probability')

dtype: float
Rows: 4
[0.5242817997932434, 0.47226759791374207, 0.47226759791374207, 0.5798847675323486]

## Visualize the prediction on a tree

In [55]:
sample_validation_data[1]

{'dti': 16.85,
 'emp_length_num': 10L,
 'grade': 'D',
 'home_ownership': 'RENT',
 'last_delinq_none': 1L,
 'last_major_derog_none': 1L,
 'purpose': 'debt_consolidation',
 'revol_util': 96.4,
 'safe_loans': 1L,
 'short_emp': 0L,
 'sub_grade': 'D1',
 'term': ' 36 months',
 'total_rec_late_fee': 0.0}

In [56]:
small_model.show(view = "Tree")

In [58]:
small_model.predict(sample_validation_data[1])

dtype: int
Rows: 1
[-1L]

## Model evaluation

In [59]:
print small_model.evaluate(train_data)['accuracy']
print decision_tree_model.evaluate(train_data)['accuracy']

0.613502041694
0.640581345369


In [60]:
print small_model.evaluate(validation_data)['accuracy']
print decision_tree_model.evaluate(validation_data)['accuracy']

0.619345109866
0.636794485136


## Complex DTC

In [61]:
big_model = graphlab.decision_tree_classifier.create(train_data, validation_set = None, target = target, features = features, max_depth = 10)

Decision tree classifier:

--------------------------------------------------------

Number of examples          : 37224

Number of classes           : 2

Number of feature columns   : 12

...

+-----------+--------------+-------------------+-------------------+

| Iteration | Elapsed Time | Training-accuracy | Training-log_loss |

+-----------+--------------+-------------------+-------------------+

| 1         | 0.064171     | 0.665538          | 0.652730          |

+-----------+--------------+-------------------+-------------------+

In [62]:
print big_model.evaluate(train_data)['accuracy']
print big_model.evaluate(validation_data)['accuracy']

0.665538362347
0.627423524343


## Evaluation

In [63]:
predictions = decision_tree_model.predict(validation_data)

In [87]:
false_positives = sum(validation_data[validation_data['safe_loans'] != predictions]['safe_loans'] == -1)
print false_positives

1656


In [89]:
false_negatives = sum(validation_data[validation_data['safe_loans'] != predictions]['safe_loans'] == 1)
print false_negatives

1716
